In [1]:
from summarization_algorithm import *
from preprocessing_algorithms import *

c:\Users\Zeyad\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import pandas as pd
data = pd.read_csv('DUC/Duc_first_reference/main_dataset/Duc_dataset_first_ref_summary.csv')
text = data.iloc[1,0]
text

"\r\nHonduras braced for potential catastrophe Tuesday as Hurricane Mitch \r\nroared through the northwest Caribbean, churning up high waves and \r\nintense rain that sent coastal residents scurrying for safer ground. \r\nPresident Carlos Flores Facusse declared a state of maximum alert \r\nand the Honduran military sent planes to pluck residents from their \r\nhomes on islands near the coast. At 0900 GMT Tuesday, Mitch was 95 \r\nmiles (152 kilometers) north of Honduras, near the Swan Islands. With \r\nwinds near 180 mph (289 kph), and even higher gusts, it was a Category \r\n5 monster _ the highest, most dangerous rating for a storm. The 350-mile \r\n(560-kilometer) wide hurricane was moving west at 8 mph (12 kph). \r\n``Mitch is closing in,'' said Monterrey Cardenas, mayor of Utila, \r\nan island 20 miles (32 kilometers) off the Honduran coast. ``And God \r\nhelp us.'' Mitch posed no immediate threat to the United States, forecasters \r\nsaid, but was expected to remain in the north

In [4]:
# # define the input text
# #
# # load the GPT-2 tokenizer and model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')
# # tokenize the input text into sentences
# sentences = text.split('. ')

# # compute the embedding of each sentence using GPT-2
# embeddings = []
# for sentence in sentences:
#     input_ids = tokenizer.encode(sentence, return_tensors='pt')
#     with torch.no_grad():
#         output = model(input_ids)[0][:, -1, :]
#         embeddings.append(output)
# # compute the similarity between each sentence embedding and the embedding of the entire text
# text_embedding = torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)
# similarity_scores = cosine_similarity(torch.cat(embeddings, dim=0), text_embedding)

# # rank the sentences based on their similarity score
# sentence_scores = list(zip(sentences, similarity_scores))
# sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

# # select the top N sentences to form the summary
# N = 5
# summary = [s[0] for s in sentence_scores[:N]]

In [5]:
# def gpt2_extractive_summarization(filtered_sentences,text, num_sentences):
#     # load the GPT-2 tokenizer and model
#     tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#     model = GPT2Model.from_pretrained('gpt2')

#     # tokenize the input text into sentences
#     sentences = text.split('. ')
#     print(sentences)

#     # compute the embedding of each sentence using GPT-2
#     embeddings = []
#     for sentence in sentences:
#         input_ids = tokenizer.encode(sentence, return_tensors='pt')
#         with torch.no_grad():
#             output = model(input_ids)[0][:, -1, :]
#             embeddings.append(output)

            

#     # compute the similarity between each sentence embedding and the embedding of the entire text
#     text_embedding = torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)
#     return embeddings,text_embedding
#     similarity_scores = cosine_similarity(torch.cat(embeddings, dim=0).numpy(), text_embedding.numpy())


#     # rank the sentences based on their similarity score
#     sentence_scores = list(zip(text, similarity_scores))
#     sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

#     # select the top N sentences to form the summary
#     summary = [s[0] for s in sentence_scores[:num_sentences]]
#     return '. '.join(summary)


In [6]:
def gpt2_extractive_summarization(sentences, filtered_sentences, size):
    # load the GPT-2 tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2Model.from_pretrained('gpt2')

    # compute the embedding of each sentence using GPT-2
    embeddings = []
    for sentence in filtered_sentences:
        input_ids = tokenizer.encode(sentence, return_tensors='pt')
        with torch.no_grad():
            output = model(input_ids)[0][:, -1, :]
            embeddings.append(output)

    # compute the similarity between each sentence embedding and the embedding of the entire text
    text_embedding = torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)
    similarity_scores = cosine_similarity(torch.cat(embeddings, dim=0).numpy(), text_embedding.numpy())

    # rank the sentences based on their similarity score
    sentence_scores = list(zip(sentences, similarity_scores))
    top_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)[:size]

    # select the top N sentences to form the summary
    summary = [s[0] for s in sentence_scores if s in top_scores]
    scores = [s[1] for s in sentence_scores if s in top_scores]
    return summary


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer and stopword list
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

# Original sentences
original_sentences = ["The quick brown fox jumps over the lazy dog.",
                       "This is a sentence.",
                       "Another sentence here.",
                       "This is a short sentence.",
                       "The dog is lazy."]

# Preprocess the sentences
filtered_sentences = []
for sentence in original_sentences:
    # Tokenize the sentence
    words = nltk.word_tokenize(sentence.lower())
    
    # Remove stopwords and lemmatize the remaining words
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords]
    
    # Convert the filtered words back to a sentence and add it to the list of filtered sentences
    filtered_sentence = ' '.join(filtered_words)
    filtered_sentences.append(filtered_sentence)

In [8]:
from preprocessing_algorithms import *

In [9]:
article_str_sentences, processed_sentences = preprocessing_text_with_spacy(text, False, False)

In [10]:
import pandas as pd
df = pd.read_csv("DUC\Duc_first_reference\main_dataset\Duc_dataset_first_ref_summary.csv")
df = df["Original Article"]

In [11]:
sentences, processed_sentences =process_one_column_df(df)

100%|██████████| 49/49 [02:03<00:00,  2.52s/it]


In [12]:
gpt2_extractive_summarization(processed_sentences,article_str_sentences,5)

[['honduras braced for potential catastrophe tuesday as hurricane mitch roared through the northwest caribbean churning up high waves and intense rain that sent coastal residents scurrying for safer ground',
  'president carlos flores facusse declared a state of maximum alert and the honduran military sent planes to pluck residents from their homes on islands near the coast',
  'at 0900 gmt tuesday mitch was 95 miles 152 kilometers north of honduras near the swan islands',
  'with winds near 180 mph 289 kph and even higher gusts it was a category 5 monster',
  'the highest most dangerous rating for a storm',
  'the 350-mile 560-kilometer wide hurricane was moving west at 8 mph 12 kph',
  'mitch is closing in said monterrey cardenas mayor of utila an island 20 miles 32 kilometers off the honduran coast',
  'and god help us',
  'mitch posed no immediate threat to the united states forecasters said but was expected to remain in the northwest caribbean for five days',
  'the u s national w

In [13]:
gpt2_extractive_summarization(processed_sentences[0],sentences[0],5)

['opposition parties led by ranariddh and former finance minister sam ram rainsy have refused to enter into a coalition with hun sen until their allegations of election fraud have been thoroughly investigated',
 'it could also hasten cambodias entry into the association of southeast asian nations put on hold last year after hun sen deposed ranariddh as co-prime minister in a bloody coup',
 'sam rainsy a virulent critic of hun sen and his partys 14 other members of parliament have remained outside cambodia since september amid fears for their security',
 'he said it contained indirect language and loopholes that suggest he and his sam rainsy party members are still under threat of arrest from hun sens ruling party',
 'the prince also said that his top general nhek bunchhay would not be given back his previous position as the second-ranking general in the cambodian militarys general staff']

In [14]:
gpt2_extractive_summarization(processed_sentences[1],sentences[1],5)

['la ceiba officials appealed for pure water for those in shelters and some residents set out plastic buckets to collect rainwater',
 'more than 20 000 tourists had abandoned cancun and nearby resort areas leaving hotels at about 20 percent of capacity',
 'guatemala reported 100 storm-related deaths',
 'six people died in southern mexico and seven in costa rica',
 'the estimated number of homeless dropped from 580 000 to 569 000 thursday']

In [16]:
list_of_summaries = []
for i in range(49):
    list_of_summaries.append(gpt2_extractive_summarization(processed_sentences[i],sentences[i],5))

In [17]:
list_of_summaries

[['opposition parties led by ranariddh and former finance minister sam ram rainsy have refused to enter into a coalition with hun sen until their allegations of election fraud have been thoroughly investigated',
  'it could also hasten cambodias entry into the association of southeast asian nations put on hold last year after hun sen deposed ranariddh as co-prime minister in a bloody coup',
  'sam rainsy a virulent critic of hun sen and his partys 14 other members of parliament have remained outside cambodia since september amid fears for their security',
  'he said it contained indirect language and loopholes that suggest he and his sam rainsy party members are still under threat of arrest from hun sens ruling party',
  'the prince also said that his top general nhek bunchhay would not be given back his previous position as the second-ranking general in the cambodian militarys general staff'],
 ['la ceiba officials appealed for pure water for those in shelters and some residents set o

In [19]:
df = pd.DataFrame(list_of_summaries)

In [20]:
df.head()

,0,1,2,3,4,5
0,opposition parties led by ranariddh and former...,it could also hasten cambodias entry into the ...,sam rainsy a virulent critic of hun sen and hi...,he said it contained indirect language and loo...,the prince also said that his top general nhek...,None
1,la ceiba officials appealed for pure water for...,more than 20 000 tourists had abandoned cancun...,guatemala reported 100 storm-related deaths,six people died in southern mexico and seven i...,the estimated number of homeless dropped from ...,None
2,the arrest warrant however referred only to qu...,the appeals argue that crimes committed during...,the reporter said he talked briefly to pinoche...,they also maintained that as a former head of ...,his lawyer clive nicholls said that if a bid t...,None
3,the taliban have refused to turn over bin lade...,but intelligence agents previously obtained ev...,the criticism is also based on the suspicion t...,officials say u s intelligence has received re...,bin laden a saudi exile is thought to be livin...,None
4,if the players win the owners will be liable f...,if the players win the owners will be liable f...,the main sticking point has been the owners in...,but neither the players nor the owners are cou...,last year more players than ever before receiv...,None


In [21]:
df = pd.DataFrame(columns=['Gpt-2'])

In [23]:
for i in range(len(list_of_summaries)):  
    df.loc[len(df)] = list_of_summaries[i]

ValueError: cannot set a row with mismatched columns

In [24]:
list_of_strings = [' '.join(inner_list) for inner_list in list_of_summaries]

In [26]:
len(list_of_strings)

49

In [27]:
df = pd.DataFrame(list_of_strings, columns=['Gpt-2'])


In [30]:
df.head()
# df.to_csv("Gpt-2_5_words_summary_with_false_false.csv")

,Gpt-2
0,opposition parties led by ranariddh and former...
1,la ceiba officials appealed for pure water for...
2,the arrest warrant however referred only to qu...
3,the taliban have refused to turn over bin lade...
4,if the players win the owners will be liable f...


In [31]:
df = pd.read_csv("Gpt-2_5_words_summary_with_false_false.csv")
df.head()

,Unnamed: 0,Gpt-2
0,0,opposition parties led by ranariddh and former...
1,1,la ceiba officials appealed for pure water for...
2,2,the arrest warrant however referred only to qu...
3,3,the taliban have refused to turn over bin lade...
4,4,if the players win the owners will be liable f...
